In [1]:
from imports import *
import warnings
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
def get_data():
    url = get_db_url('superstore_db')
    sql = '''
          SELECT ord.*,
                 cat.Category,
                 prod.`Product Name`,
                 cust.`Customer Name`,
                 reg.`Region Name`
              FROM orders ord
                LEFT JOIN categories cat USING(`Category ID`)
                LEFT JOIN products prod USING(`Product ID`)
                LEFT JOIN customers cust USING(`Customer ID`)
                LEFT JOIN regions reg USING(`Region ID`)
          '''
    df = pd.read_sql(sql, url)
    return df

In [3]:
df = get_data()

In [4]:
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,Product ID,Sales,Quantity,Discount,Profit,Category ID,Region ID,Category,Product Name,Customer Name,Region Name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-BO-10001798,261.9600,2.0,0.00,41.9136,1,1,Furniture,Bush Somerset Collection Bookcase,Claire Gute,South
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-CH-10000454,731.9400,3.0,0.00,219.5820,2,1,Furniture,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",Claire Gute,South
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,OFF-LA-10000240,14.6200,2.0,0.00,6.8714,3,2,Office Supplies,Self-Adhesive Address Labels for Typewriters by Universal,Darrin Van Huff,West
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,FUR-TA-10000577,957.5775,5.0,0.45,-383.0310,4,1,Furniture,Bretford CR4500 Series Slim Rectangular Table,Sean O'Donnell,South
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,OFF-ST-10000760,22.3680,2.0,0.20,2.5164,5,1,Office Supplies,Eldon Fold 'N Roll Cart System,Sean O'Donnell,South


In [5]:
def prep_data(df):
    
    # rename columns: lower case, remove space
    for col in df.columns:
        df = df.rename(columns={col: col.lower().replace(' ', '_')})
        
    # cast date columns as datetime type
    df.order_date = pd.to_datetime(df.order_date)
    df.ship_date = pd.to_datetime(df.ship_date)
        
    # make datetime index
    df.index = df.order_date
    
    # add profit per product column
    df['profit_per_product'] = df.profit / df.quantity
    
    # add sales per product
    df['sales_per_product'] = df.sales / df.quantity

    return df

In [6]:
df = prep_data(df)

In [7]:
df.head()

,order_id,order_date,ship_date,ship_mode,customer_id,segment,country,city,state,postal_code,product_id,sales,quantity,discount,profit,category_id,region_id,category,product_name,customer_name,region_name,profit_per_product,sales_per_product,month,weekday
order_date,,,,,,,,,,,,,,,,,,,,,,,,,
2016-11-08,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-BO-10001798,261.9600,2.0,0.00,41.9136,1,1,Furniture,Bush Somerset Collection Bookcase,Claire Gute,South,20.9568,130.9800,11,1-Tue
2016-11-08,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-CH-10000454,731.9400,3.0,0.00,219.5820,2,1,Furniture,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",Claire Gute,South,73.1940,243.9800,11,1-Tue
2016-06-12,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,OFF-LA-10000240,14.6200,2.0,0.00,6.8714,3,2,Office Supplies,Self-Adhesive Address Labels for Typewriters by Universal,Darrin Van Huff,West,3.4357,7.3100,6,6-Sun
2015-10-11,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,FUR-TA-10000577,957.5775,5.0,0.45,-383.0310,4,1,Furniture,Bretford CR4500 Series Slim Rectangular Table,Sean O'Donnell,South,-76.6062,191.5155,10,6-Sun
2015-10-11,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,OFF-ST-10000760,22.3680,2.0,0.20,2.5164,5,1,Office Supplies,Eldon Fold 'N Roll Cart System,Sean O'Donnell,South,1.2582,11.1840,10,6-Sun


In [8]:
def split_data(df):
    '''
    Splits data into train and test based on year.
    '''
    train = df['2014':'2016']
    test = df['2017']
    return train, test

In [9]:
train, test = split_data(df)

In [10]:
train.shape

(1238, 25)

In [11]:
test.shape

(496, 25)